In [1]:
%load_ext autoreload
%autoreload 2

import json
import logging
import os
import re
import sys
from copy import deepcopy
from functools import cache, cached_property, partial
from inspect import signature
from itertools import chain
from pathlib import Path
from uuid import uuid4

import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import QuantLib as ql
import talib as ta
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from gioutils import ezutils as ez
from gioutils.blpw import BlpQuery
from gioutils.ezutils import bh, bhs
from gioutils.gui.pandas import millify
from gioutils.utils import (
    RepeatedTimer,
    date_from_offset,
    get_bbg_fut_chain_ticker,
    get_bday,
    get_n_from_code,
    get_next_n,
    parse_offset,
    parse_swap_tenor_expr,
    today,
    to_pickle,
    read_pickle
)
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from joblib import Parallel, delayed
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import ParameterGrid
from techi import api as ti
from termcolor import colored

from api.data.base import data_path, radar_path
from api.gui.pandas import DataGridGio, DFOutput, custom_styler_row_line, negative_red
from api.gui.params import bbg_params, fwd_params, spot_params
from api.gui.theme import bg_color, grid_style
from api.instruments import config as c
from api.instruments.cache import data_cache, get_key
from api.instruments.future import FutureChain, FutureFly, FutureSpread
from api.instruments.signals import EnsambleSignal, MACDHistCrossover, ZScoreThreshold
from api.instruments.signals2 import BaseSignal, MACDCustomSignal, get_crossover_signal
from api.instruments.strategies import FutureStrategy
from api.instruments.strategies2 import MACDCustomStrategy
from api.instruments.swap import Fly, RealSwap, RollingSwap, Spread
from api.instruments.utils import ez_bump, parse_dates, parse_number


# from bhsysapi import BHSysApi

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None)
c.USE_T_COST = True

bq = BlpQuery(timeout=50000).start()
# ba = BHSysApi(password="A&3jg5Aj")

notional = 1000
hist = "10y"
strategy_hist = "1y"

## Instruments Parameters

rates = [
    "er",
    "sfr",
    "ff",
    "sfi",
]

govt = ["tu", "fv", "ty", "us", "du", "oe", "rx", "ub", "ik"]

rates_single_legs = range(1, 12 + 1)
govt_single_legs = range(1, 2 + 1)

fly_legs = [
    (2, 3, 4),
    (2, 4, 6),
    (2, 5, 8),
    (2, 6, 10),
    (2, 7, 12),
    (3, 4, 5),
    (3, 5, 7),
    (3, 6, 9),
    (3, 7, 11),
    (3, 8, 13),
    (4, 6, 8),
    (4, 7, 9),
    (4, 7, 11),
    (4, 8, 12),
    (5, 8, 11),
    (5, 9, 13),
    (6, 9, 12),
]

spread_legs = [
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (2, 9),
    (3, 5),
    (3, 6),
    (3, 7),
    (3, 8),
    (3, 9),
    (3, 10),
    (3, 11),
    (3, 12),
    (4, 6),
    (4, 7),
    (4, 8),
    (4, 9),
    (4, 10),
    (4, 11),
    (4, 12),
    (5, 8),
    (5, 9),
    (5, 10),
    (5, 11),
    (5, 12),
    (6, 8),
    (6, 9),
    (6, 10),
    (6, 11),
    (6, 12),
    (7, 10),
    (7, 11),
    (7, 12),
    (8, 12),
]

watching = {"du1", "oe1", "rx1", "ik1", "er1"}

watching = {
    'du1': 1,
    'oe1': 1,
    'rx1': 1,
    'ub1': 2,
    'ik1': 1,
    "er1": 2,
    "er2": 1,
    "er3": 1,
    "er4": 1,
    "er5": 1,
    "er6": 1,
    "er7": 1,
    "er8": 1,
    "er9": 2,
    "er10": 3,
    "er11": 3,
    "er12": 1,
    "sfr1": 3,
    "sfr2": 2,
    "sfr3": 1,
    "sfr4": 2,
    "sfr5": 2,
    "sfr6": 1,
    "sfr7": 1,
    "sfr8": 2,
    "sfr9": 2,
    "sfr10": 2,
    "sfr11": 2,
    "sfr12": 1,
    "ff1": 2,
    "ff2": 3,
    "ff3": 3,
    "ff4": 1,
    "ff5": 1,
    "ff6": 2,
    "ff7": 2,
    "ff8": 1,
    "ff9": 2,
    "ff10": 2,
    "ff11": 2,
    "ff12": 2,
    "sfi1": 2,
    "sfi2": 3,
    "sfi3": 2,
    "sfi4": 1,
    "sfi5": 1,
    "sfi6": 1,
    "sfi7": 1,
    "sfi8": 1,
    "sfi9": 2,
    "sfi10":2,
    "sfi11": 3,
    "sfi12": 2,
}

stir_flies = read_pickle('top_stir_flies.pickle')

ress = {**stir_flies}

strategies = {k: deepcopy(ss.iloc[0]) for k, ss in ress.items()}

# all_instruments = {s.instrument.ticker: s.instrument for k, s in strategies.items()}
# setting the history
start_hist = '10y'
end_hist = today()
all_instruments = {}
for k, strategy in strategies.items():
    for instrument in strategy.instrument.instruments:
        instrument.start_hist = start_hist
        instrument.end_hist = end_hist
        all_instruments.setdefault(instrument.ticker, instrument)

fields = {
    'px_last': 'quote', 
    'yld_ytm_mid': 'yld'
}

ref_fields = {
    'fut_tick_size': 'tick_size',
    'fut_tick_val': 'tick_value', 
    'fut_val_pt': 'point_value'
    
}

symbols = {instrument.symbol: instrument for key, instrument in all_instruments.items()}
# keys = [get_key(s.instrument, attr, keys=['start_hist', 'end_hist']) for key, s in strategies.items() for attr in ['quote', 'yld']]

# History download
hist = bq.bdh(
    securities=[*symbols.keys()],
    fields=fields,
    start_date=today() - parse_offset(start_hist),
)
hist = hist.set_index('date')

# Updating history
for symbol, instrument in symbols.items():
    for attr in fields.values():
        series = hist.query(f'security == "{symbol}"')[attr]
        series = instrument._rename_attr(series, attr)
        key = get_key(instrument, attr, keys=['start_hist', 'end_hist'])
        data_cache[key] = series

# Reference download
ref = bq.bdp(
    securities=[*symbols.keys()],
    fields=ref_fields,
)

# Updating reference
for symbol, instrument in symbols.items():
    for attr in ref_fields.values():
        val = ref.query(f'security == "{symbol}"')[attr].squeeze()
        try:
            val = float(val)
        except:
            pass
        key = get_key(instrument, attr)
        data_cache[key] = val

# Running strats for entire history
for key, self in strategies.items():
    #print(key)
    self.run()

# Last donwload
last = bq.bdp(
    securities=[*symbols.keys()],
    fields=fields,
)

# Updating last
for symbol, instrument in symbols.items():
    for attr in fields.values():
        value = last.query(f'security == "{symbol}"')[attr].squeeze()
        getattr(instrument, attr).iloc[-1] = value

# Running last
for key, self in strategies.items():
    # print(key)
    self.run(
        start=-1
    )

watch_codes = [
    1, 
    # 2, 
    # 3
]

for key, self in strategies.items():
    if (self._running or self._pnl.index[-1] == today()) and watching.get(key) in watch_codes:
        display(key.upper())
        last_trade = self._summary['trade'].iloc[-1]
        display(self._pnl.query(f"trade == {last_trade}"))

i = 0
for k, v in strategies.items():
    if v._running:
        i+= 1
        print(k, v.instrument.quote.iloc[-1:])
        if i == 3:
            break

v._summary

for key, self in strategies.items():
    if self._running:
        last_trade = self._summary['trade'].iloc[-1]
        display(v.Html(children=key.upper(), tag='h2'))
        out = w.Output()
        out_chart = w.Output()
        with out:
            display(self._pnl.query(f"trade == {last_trade}"))
        with out_chart:
            display(self._pnl['pv'].plot())
        display(w.VBox([out , out_chart]))

for key, self in strategies.items():

    last_trade = self._summary['trade'].iloc[-1]
    display(v.Html(children=key.upper(), tag='h2'))
    out = w.Output()
    with out:
        display(self._pnl.query(f"trade == {last_trade}"))
    display(w.VBox([out , go.FigureWidget(self._pnl['pv'].plot())]))

for key, self in strategies.items():
    if self._running:
        display(key.upper())
        last_trade = self._summary['trade'].iloc[-1]
        display(self._pnl.query(f"trade == {last_trade}"))

strategies['sfi (2, 5, 8)']._pnl